# Testing Skysurvey

In [139]:
import skysurvey
from skysurvey import effects

#snia = skysurvey.SNeIa()
snia = skysurvey.SNeIa.from_draw(1000, effect=effects.mw_extinction)
data = snia.draw(size=50_000, tstart=56_000, tstop=56_100, inplace=True) # see options


data.head(5) # also snia.data

,z,x1,c,t0,ra,dec,magabs,mwebv,magobs,x0,template
0,0.10725,-1.670,-0.089,56094.722656,6.745185,50.974640,-19.357897,0.201597,19.191687,0.000334,salt2
1,0.09665,-0.380,0.035,56052.453125,134.031586,-12.526664,-19.077568,0.089938,19.230837,0.000322,salt2
2,0.12975,0.420,0.087,56044.738281,80.426414,30.075794,-19.062799,0.391901,19.931856,0.000169,salt2
3,0.14435,-0.405,0.213,56022.117188,152.932999,0.157164,-18.704332,0.036440,20.541794,0.000096,salt2
4,0.19745,-2.010,-0.054,56084.414062,304.096069,-3.050796,-19.205429,0.144299,20.790045,0.000077,salt2


In [141]:
import numpy as np
from skysurvey.tools import utils

size = 10_000

# footprint
from shapely import geometry
sq_footprint = geometry.box(-1, -1, +1, +1)

# Observing data
ra, dec = utils.random_radec(size=size, ra_range=[200,250], dec_range=[-20,10])

data = {}
data["ra"] = ra
data["dec"] = dec
data["gain"] = 1
data["zp"] = 30
data["skynoise"] = np.random.normal(size=size, loc=150, scale=20)
data["mjd"] = np.random.uniform(56_000-10, 56_100 + 10, size=size)
data["band"] = np.random.choice(["desg","desr","desi"], size=size)

# Build the survey
survey = skysurvey.Survey.from_pointings(data, footprint=sq_footprint)
survey.data

,ra,dec,gain,zp,skynoise,mjd,band,fieldid_survey,fieldid
218828,212.607254,-4.655582,1,30,141.150894,55990.019531,desg,2376,256325
218875,212.607254,-4.655582,1,30,141.150894,55990.019531,desg,2376,256326
218918,212.607254,-4.655582,1,30,141.150894,55990.019531,desg,2376,256327
218926,212.607254,-4.655582,1,30,141.150894,55990.019531,desg,2376,256328
218964,212.607254,-4.655582,1,30,141.150894,55990.019531,desg,2376,256329
...,...,...,...,...,...,...,...,...,...
80704,227.147552,5.715752,1,30,145.072357,56109.996094,desg,1106,220293
80726,227.147552,5.715752,1,30,145.072357,56109.996094,desg,1106,220294
80748,227.147552,5.715752,1,30,145.072357,56109.996094,desg,1106,220295
80777,227.147552,5.715752,1,30,145.072357,56109.996094,desg,1106,220296


In [143]:
from skysurvey import dataset
dset = dataset.DataSet.from_targets_and_survey(snia, survey)
dset.data

/Users/andradenebula/Documents/Research/Transient_Metrics/Rubin_Data_Preview_SNe/skysurvey-env/lib/python3.12/site-packages/skysurvey/dataset.py:726: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  this_survey = gsurvey_indexed.get_group(index_).copy()


fieldid  band           mjd  zp zpsys  gain    skynoise  \
index index_obs                                                            
36368 45               6  desr  56012.738281  30    ab     1  148.540604   
45370 59              16  desg  56025.695312  30    ab     1  148.703583   
23111 67              20  desr  56000.449219  30    ab     1  139.410568   
      68              20  desg  56003.109375  30    ab     1  143.811661   
44514 179             22  desr  56008.128906  30    ab     1  107.934235   
...                  ...   ...           ...  ..   ...   ...         ...   
30127 465237       19476  desg  56010.144531  30    ab     1  202.295319   
      465238       19476  desr  56030.898438  30    ab     1  157.288193   
      465239       19476  desr  56092.902344  30    ab     1  183.142273   
42526 465260       19478  desg  55992.621094  30    ab     1  172.114441   
47415 465261       19491  desg  55992.621094  30    ab     1  172.114441   

                        flux     fluxerr  
index index_obs                           
36368 45          -33.958267  148.540602  
45370 59          120.967658  148.703584  
23111 67          229.970620  139.410569  
      68          169.582035  143.811658  
44514 179         238.421183  107.934234  
...                      ...         ...  
30127 465237     3294.705516  211.010749  
      465238     1818.460876  163.457230  
      465239      211.826707  183.142277  
42526 465260      -57.554207  172.114441  
47415 465261      205.022708  172.114441  

[48429 rows x 9 columns]